In [1]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [10]:
# Driver's path
path = 'C:/Users/papan/Downloads/chromedriver_win32'
driver = webdriver.Chrome(path)
# Maximize Window
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)
# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)
# Accept cookies
driver.find_element('xpath','//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]').click()
# User Credentials
# Reading txt file where we have our user credentials
with open('user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]
user_name = user_credentials[0] # First line
password = user_credentials[1] # Second line
driver.find_element('xpath','//*[@id="username"]').send_keys(user_name)
driver.find_element('xpath','//*[@id="password"]').send_keys(password)
time.sleep(1)
# Login button
driver.find_element('xpath','//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)
# if driver.find_element('xpath','//*[@id="ember7"]/h2').text == 'Confirm your account information':
#     driver.find_element('xpath','//*[@id="ember20"]/button[1]').click()
driver.find_element('xpath','//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(5)
driver.execute_script("return window.stop();")
jobs_block = driver.find_elements(By.ID,'jobs-home-vertical-list__entity-list')
links = []
for block in jobs_block:
    jobs_list = block.find_elements(By.TAG_NAME, 'li')
    driver.implicitly_wait(10)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.TAG_NAME, "a")))
    for job in jobs_list:
        innerHTML=job.get_attribute('innerHTML')
        soup = BeautifulSoup(innerHTML,'html')
        link = soup.find('a',{'class':'ember-view job-card-container__link job-card-list__title'},href=True)
        if link:
            links.append(link['href'])

C:\Users\papan\AppData\Local\Temp\ipykernel_1544\3097825942.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [11]:
links

['/jobs/collections/recommended/?currentJobId=3590306565&eBP=JYMBII_JOBS_HOME_ORGANIC&refId=7xvaqrjlS9GA93wS%2F1q07w%3D%3D&trackingId=Kg8yAEKcA1MoSBJtrDPDjw%3D%3D',
 '/jobs/collections/recommended/?currentJobId=3598111263&eBP=CwEAAAGIDFmBmM_0OYjNJOT7PBvoLJnSWJEjGp8zxD5b3EWF0ZZgV9wIl8u16WcTuCVFSbdCYVu8FZXsduduEGsqNYRf9FBqEX0-vEDBSaWDqkO8uBnU_jo440j0QNEq_3pnZaOFTrLhF965jEqEoe4W6W6cCyU-a_itsqC2Px-iz4lM7ymn72gulY6WGiBMnVHjK9MmmvyC7idLg_SuOnFcvMTQuHcYMo2NfLkAz2AqenzvNKh6Hu_YnOj4RDJa-it_0gcSWmd7kM6Htcyh7OLtebcphU_NRo6bBi4RRxedf7x-2nqWw3wVzoGXPhNz4OCR8ur7S1qxzsubSmv1Fpd9n7rP38_tUfU8KPmIUoBRlDE&refId=7xvaqrjlS9GA93wS%2F1q07w%3D%3D&trackingId=LmMOyWayOOGkxoqY4vroLA%3D%3D',
 '/jobs/collections/recommended/?currentJobId=3599846072&eBP=CwEAAAGIDFmBmND7bGAI2EVv2MqUfiVzY2CkGLTza-SY2L83A17aUtfuEjoAfkc6fxw0Rx1xX19Z8J5oOhLl2oqC5bFg6Qa8naNUDgIKaE0mckEWgzvlRBj0hVbHBaJeOfN0T7w-_iWkRitoCHpYpkticp_uKwopLyAZwMP6t09_0mjE0C2f3-DFo3sxbc-OdzIO0go2To3m64vFOIecmernxy4jjurtVfldkkAJ32EeI4JZ5I3qAnw74-cM5BNQbNZiWeFJOn

In [39]:
len(jobs_list)

11